In [1]:
import numpy as np
import random
import optuna
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 1234
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"

In [3]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)

In [4]:
def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        digits=4
    )

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    Returns only the F1-score for trend change.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

In [5]:
# Cargar datos
train = pd.read_csv("../../../data/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/validation_set.csv", parse_dates=["date"])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values

IMPORTANTE: Hay que sumar a la columna de prediccion porque -1 no funciona en funcion de LOSS

In [6]:
y_train += 1
y_val += 1

In [8]:
# Optuna para hiperparámetros
def objective(trial):
    set_seeds(SEED)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 16)
    n_estimators = trial.suggest_int("n_estimators", 100, 500)
    subsample = trial.suggest_float("subsample", 0.6, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 2.0)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 2.0)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 10)

    model = XGBClassifier(
        learning_rate=learning_rate,
        max_depth=max_depth,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        min_child_weight=min_child_weight,
        random_state=SEED,
        eval_metric="mlogloss"
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = trend_changes_true(y_val, y_pred)
    return score

In [9]:
set_seeds(SEED)
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=100)

[I 2025-08-17 22:31:47,413] A new study created in memory with name: no-name-4cb1df61-87be-462c-9693-5ef0fd3469c5
[I 2025-08-17 22:31:48,350] Trial 0 finished with value: 0.1 and parameters: {'learning_rate': 0.01774894524799066, 'max_depth': 11, 'n_estimators': 275, 'subsample': 0.9141434334855076, 'colsample_bytree': 0.8899879040594018, 'reg_lambda': 0.5451852105652832, 'reg_alpha': 0.5529285102861934, 'min_child_weight': 9}. Best is trial 0 with value: 0.1.
[I 2025-08-17 22:31:48,830] Trial 1 finished with value: 0.23809523809523808 and parameters: {'learning_rate': 0.17642821458092559, 'max_depth': 15, 'n_estimators': 243, 'subsample': 0.8003980502093835, 'colsample_bytree': 0.8417314675860681, 'reg_lambda': 1.4254040539658004, 'reg_alpha': 0.7405015095807899, 'min_child_weight': 6}. Best is trial 1 with value: 0.23809523809523808.
[I 2025-08-17 22:31:49,944] Trial 2 finished with value: 0.23809523809523808 and parameters: {'learning_rate': 0.045136334609244884, 'max_depth': 2, 'n_

In [10]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de {SCORE}: {study.best_value:.4f}")

Mejores hiperparámetros encontrados:
{'learning_rate': 0.10068445706066761, 'max_depth': 3, 'n_estimators': 350, 'subsample': 0.9007890427902547, 'colsample_bytree': 0.7314800550236931, 'reg_lambda': 1.9823224974072524, 'reg_alpha': 1.5821543506396027, 'min_child_weight': 9}
Mejor score de f1-score: 0.3415


In [12]:
# Entrenar modelo final con los mejores hiperparámetros
set_seeds(SEED)
best_params = study.best_params
final_model = XGBClassifier(
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    n_estimators=best_params["n_estimators"],
    subsample=best_params["subsample"],
    colsample_bytree=best_params["colsample_bytree"],
    reg_lambda=best_params["reg_lambda"],
    reg_alpha=best_params["reg_alpha"],
    min_child_weight=best_params["min_child_weight"],
    random_state=SEED,
    eval_metric="mlogloss"
)
final_model.fit(X_train, y_train)
y_pred_final = final_model.predict(X_val)
print("\nXGBClassifier FINAL trend_changes_score:\n", trend_changes_score(y_val, y_pred_final))


XGBClassifier FINAL trend_changes_score:
               precision    recall  f1-score   support

       False     0.9048    0.8482    0.8756       112
        True     0.2917    0.4118    0.3415        17

    accuracy                         0.7907       129
   macro avg     0.5982    0.6300    0.6085       129
weighted avg     0.8240    0.7907    0.8052       129

